# Li(110) Step-Edge Adatom Diffusion – Restartable NEB (JSON + Visualisation)

This notebook follows the same pattern as your multi-material restartable NEB:

- Tracks progress in a `status.json` file.
- Uses `.traj` files for relaxed states and NEB images.
- Logs every optimisation / NEB step (max force + elapsed time).
- **Additionally**, it *visually shows*:
  - the step-edge slab,
  - the **unrelaxed initial** configuration,
  - the **unrelaxed final** configuration,

so you can check the geometry before running expensive GPAW + NEB.

The only structural difference from the flat-surface version is the surface construction:
we build a Li(110) **step-edge slab** and place the adatom on upper/lower terraces away
from the simulation cell edges.


In [1]:
# 0. Imports, NGLView setup, and global configuration

from ase.build import bcc110
from ase.io import read, write, Trajectory
from ase import Atom
from ase.optimize import BFGS, FIRE
from ase.mep import NEB
from gpaw import GPAW, LCAO
from ase.constraints import FixAtoms

import numpy as np
import os
import json
import time

import nglview as nv


print("Imports OK, NGLView enabled.")

# Global settings
a0 = 3.49  # Li bcc lattice constant (adjust if needed)

# Practical-size slab
size = (4, 4, 6)   # (nx, ny, nz), ny must be even for orthogonal=True
vacuum = 8.0

adatom_height = 2.0
edge_fraction = 0.3  # 0.3 → closer to step, 0.5 → halfway to terrace centre

neb_n_images = 5     # total images including endpoints
neb_fmax = 0.10
neb_max_steps = 300

output_dir = "Li110_step_edge_neb_restartable_json"
os.makedirs(output_dir, exist_ok=True)

status_path = os.path.join(output_dir, "status.json")
print(f"Output directory: {output_dir}")
print(f"Status file: {status_path}")


Imports OK, NGLView enabled.
Output directory: Li110_step_edge_neb_restartable_json
Status file: Li110_step_edge_neb_restartable_json/status.json


In [2]:
# 1. JSON status helpers

def load_status():
    if not os.path.exists(status_path):
        return {}
    try:
        with open(status_path, "r") as f:
            return json.load(f)
    except Exception:
        return {}


def save_status(status):
    with open(status_path, "w") as f:
        json.dump(status, f, indent=2)
        f.flush()
    return status

print("Status helpers defined.")


Status helpers defined.


In [3]:
# 2. Build Li(110) step-edge slab and compute step metadata

def build_Li110_step_slab():
    """Build an orthogonal Li(110) slab and carve a monoatomic step.

    Returns
    -------
    slab : Atoms
        Step-edge slab.
    meta : dict
        Dictionary with:
        - z_unique: list of z-layer values
        - z_top: topmost layer z
        - z_second: layer just below z_top
        - y_cut: y-position of step edge
    """
    slab_flat = bcc110('Li', a=a0, size=size, vacuum=vacuum, orthogonal=True)
    slab_flat.center(axis=2)

    pos = slab_flat.get_positions()
    x = pos[:, 0]
    y = pos[:, 1]
    z = pos[:, 2]

    # Identify distinct z-layers
    z_unique = np.unique(np.round(z, 5))
    z_unique.sort()
    z_top = float(z_unique[-1])        # topmost layer
    z_second = float(z_unique[-2])     # layer just below

    tol = 1e-3
    mask_top = np.abs(z - z_top) < tol

    # Define step edge by a plane in y (left vs right half of surface)
    y_cut = float(np.mean(y[mask_top]))

    # Remove top-layer atoms on "left" half (y < y_cut) → creates step
    keep_mask = ~(mask_top & (y < y_cut))
    step_slab = slab_flat[keep_mask]
    step_slab.center(axis=2)

    meta = {
        "z_unique": z_unique.tolist(),
        "z_top": z_top,
        "z_second": z_second,
        "y_cut": y_cut,
    }
    return step_slab, meta

print("Step-slab builder defined.")


Step-slab builder defined.


In [4]:
# 3. Visualise step-edge slab and unrelaxed initial/final (for inspection)

# Build a preview slab and metadata
step_slab_preview, meta_preview = build_Li110_step_slab()
z_top_prev = meta_preview["z_top"]
z_second_prev = meta_preview["z_second"]
y_cut_prev = meta_preview["y_cut"]

print("Preview step slab:")
print("z layers:", meta_preview["z_unique"])
print(f"z_top = {z_top_prev:.4f}, z_second = {z_second_prev:.4f}")
print(f"y_cut = {y_cut_prev:.4f}")

write(os.path.join(output_dir, "Li110_step_slab_preview.xyz"), step_slab_preview)

view_step = nv.show_ase(step_slab_preview)
view_step.add_unitcell()
view_step


Preview step slab:
z layers: [8.0, 10.4678, 12.93561, 15.40341, 17.87121, 20.33901]
z_top = 20.3390, z_second = 17.8712
y_cut = 3.7017


/Users/tew/Documents/Project/GitHub/chem-com/.venv/lib/python3.13/site-packages/ase/io/extxyz.py:318: UserWarning: Skipping unhashable information adsorbate_info
  warnings.warn('Skipping unhashable information '


NGLWidget()

In [5]:
# 4. Adatom placement helpers – centred away from box edges

def make_initial_step_state(step_slab, z_top, y_cut, height=adatom_height, edge_fraction=edge_fraction):
    """Adatom on UPPER terrace near step edge but shifted toward interior."""
    atoms = step_slab.copy()
    pos = atoms.get_positions()
    x, y, z = pos[:,0], pos[:,1], pos[:,2]

    tol = 1e-3
    # upper terrace = top layer, y > y_cut
    mask_upper = (np.abs(z - z_top) < tol) & (y > y_cut)
    if not np.any(mask_upper):
        raise RuntimeError("No atoms found on upper terrace (check y_cut / z_top).")

    x_u = x[mask_upper]
    y_u = y[mask_upper]

    x_min, x_max = x_u.min(), x_u.max()
    y_min, y_max = y_u.min(), y_u.max()

    # x at centre of terrace
    x_new = 0.5 * (x_min + x_max)

    # y_new between step edge and terrace interior
    y_target = 0.5 * (y_min + y_max)
    y_new = y_cut + edge_fraction * (y_target - y_cut)

    z_new = z_top + height

    atoms.append(Atom('Li', (x_new, y_new, z_new)))
    return atoms


def make_final_step_state(step_slab, z_second, y_cut, height=adatom_height, edge_fraction=edge_fraction):
    """Adatom on LOWER terrace near step edge but shifted toward interior."""
    atoms = step_slab.copy()
    pos = atoms.get_positions()
    x, y, z = pos[:,0], pos[:,1], pos[:,2]

    tol = 1e-3
    # lower terrace = second layer, y < y_cut
    mask_lower = (np.abs(z - z_second) < tol) & (y < y_cut)
    if not np.any(mask_lower):
        raise RuntimeError("No atoms found on lower terrace (check y_cut / z_second).")

    x_l = x[mask_lower]
    y_l = y[mask_lower]

    x_min, x_max = x_l.min(), x_l.max()
    y_min, y_max = y_l.min(), y_l.max()

    # x at centre
    x_new = 0.5 * (x_min + x_max)

    # y_new between step edge and terrace interior (y_target < y_cut)
    y_target = 0.5 * (y_min + y_max)
    y_new = y_cut + edge_fraction * (y_target - y_cut)

    z_new = z_second + height

    atoms.append(Atom('Li', (x_new, y_new, z_new)))
    return atoms

print("Adatom placement helpers defined.")


Adatom placement helpers defined.


In [6]:
# 5. Visualise UNRELAXED initial and final configurations (preview)

initial_unrelaxed_preview = make_initial_step_state(step_slab_preview,
                                                    z_top_prev,
                                                    y_cut_prev,
                                                    height=adatom_height,
                                                    edge_fraction=edge_fraction)

final_unrelaxed_preview = make_final_step_state(step_slab_preview,
                                                z_second_prev,
                                                y_cut_prev,
                                                height=adatom_height,
                                                edge_fraction=edge_fraction)

write(os.path.join(output_dir, "Li110_initial_unrelaxed_preview.xyz"), initial_unrelaxed_preview)
write(os.path.join(output_dir, "Li110_final_unrelaxed_preview.xyz"), final_unrelaxed_preview)

print("Showing unrelaxed INITIAL configuration:")
view_init_prev = nv.show_ase(initial_unrelaxed_preview)
view_init_prev.add_unitcell()
display(view_init_prev)

print("Showing unrelaxed FINAL configuration:")
view_final_prev = nv.show_ase(final_unrelaxed_preview)
view_final_prev.add_unitcell()
view_final_prev


Showing unrelaxed INITIAL configuration:


NGLWidget()

Showing unrelaxed FINAL configuration:


NGLWidget()

In [7]:
# 6. GPAW helper and relaxation with logged steps (restartable)

def apply_bottom_fix(slab, fraction=0.5):
    z = slab.get_positions()[:, 2]
    z_min, z_max = z.min(), z.max()
    z_cut = z_min + fraction * (z_max - z_min)
    mask = z < z_cut
    slab.set_constraint(FixAtoms(mask=mask))
    return slab


def make_gpaw(label):
    return GPAW(
        mode=LCAO(),
        xc="PBE",
        kpts=(2, 2, 1),  # relax to (2,1,1) or (1,1,1) for quick tests
        occupations={'name': 'fermi-dirac', 'width': 0.1},
        txt=os.path.join(output_dir, f"{label}.txt"),
    )


def relax_structure(atoms, label, fmax, allow_restart=True):
    """Relax a structure with BFGS until max force < fmax.

    - Writes a `<label>.traj` file to disk.
    - If `allow_restart` and traj exists, restart from last frame.
    - Logs every step with max force and elapsed time.
    """
    traj_path = os.path.join(output_dir, f"{label}.traj")

    if allow_restart and os.path.exists(traj_path):
        print(f"[{label}] Restarting from existing trajectory: {traj_path}")
        try:
            atoms = read(traj_path, index=-1)
        except Exception as e:
            print(f"[{label}] Failed to read existing traj ({e}), starting from input geometry.")

    atoms = apply_bottom_fix(atoms, fraction=0.5)
    atoms.calc = make_gpaw(label)

    dyn = BFGS(atoms,
               trajectory=traj_path,
               logfile=os.path.join(output_dir, f"{label}_opt.log"))

    step = 0
    t0 = time.time()
    for _ in dyn.irun(fmax=fmax):
        step += 1
        elapsed = time.time() - t0
        forces = atoms.get_forces()
        maxf = float(np.abs(forces).max())
        print(f"[{label}] step {step:4d}, max |F| = {maxf:.3f} eV/Å, elapsed = {elapsed:7.1f} s")

    total = time.time() - t0
    energy = atoms.get_potential_energy()
    print(f"[{label}] Relaxation finished in {step} steps, total time = {total:.1f} s, E = {energy:.6f} eV")

    return atoms, energy

print("GPAW + relax_structure defined.")


GPAW + relax_structure defined.


In [8]:
# 7. Restartable NEB driver for Li(110) step-edge diffusion

def run_Li110_step_neb(description="Li(110) step-edge descent"):
    """Run NEB for Li adatom descending a (110) step edge.

    Uses:
      - status.json to track stages,
      - .traj files for relaxed states and NEB path,
      - logged steps for relaxations and NEB iterations.
    """
    symbol = "Li"
    status = load_status()
    mat_status = status.get(symbol, {
        "initial_relaxed": False,
        "final_relaxed": False,
        "neb_done": False,
    })

    print("\n" + "=" * 60)
    print(f"Running NEB for {symbol}: {description}")
    print("Current status:", mat_status)
    print("=" * 60)

    # 1. Build step slab and metadata
    step_slab, meta = build_Li110_step_slab()
    z_top = meta["z_top"]
    z_second = meta["z_second"]
    y_cut = meta["y_cut"]

    write(os.path.join(output_dir, f"{symbol}-step_slab.xyz"), step_slab)

    # 2. Construct initial and final states (unrelaxed)
    init_path = os.path.join(output_dir, f"{symbol}-initial_unrelaxed.traj")
    final_path = os.path.join(output_dir, f"{symbol}-final_unrelaxed.traj")

    if os.path.exists(init_path) and os.path.exists(final_path):
        print(f"{symbol}: loading existing unrelaxed initial/final from traj.")
        initial = read(init_path, index=-1)
        final = read(final_path, index=-1)
    else:
        print(f"{symbol}: creating unrelaxed initial/final on step edge.")
        initial = make_initial_step_state(step_slab, z_top, y_cut,
                                          height=adatom_height,
                                          edge_fraction=edge_fraction)
        final = make_final_step_state(step_slab, z_second, y_cut,
                                      height=adatom_height,
                                      edge_fraction=edge_fraction)
        write(init_path, initial)
        write(final_path, final)

    # 3. Relax initial and final (restartable)
    if not mat_status.get("initial_relaxed", False):
        print(f"{symbol}: relaxing initial adatom configuration...")
        initial_relaxed, E_i = relax_structure(initial, f"{symbol}-initial", fmax=0.05, allow_restart=True)
        mat_status["initial_relaxed"] = True
        status[symbol] = mat_status
        save_status(status)
    else:
        print(f"{symbol}: initial state already relaxed (skipping).")
        initial_relaxed = read(os.path.join(output_dir, f"{symbol}-initial.traj"), index=-1)

    if not mat_status.get("final_relaxed", False):
        print(f"{symbol}: relaxing final adatom configuration...")
        final_relaxed, E_f = relax_structure(final, f"{symbol}-final", fmax=0.05, allow_restart=True)
        mat_status["final_relaxed"] = True
        status[symbol] = mat_status
        save_status(status)
    else:
        print(f"{symbol}: final state already relaxed (skipping).")
        final_relaxed = read(os.path.join(output_dir, f"{symbol}-final.traj"), index=-1)

    # 4. NEB (restart-aware)
    energies_file = os.path.join(output_dir, f"{symbol}-NEB-energies.dat")
    neb_traj_path = os.path.join(output_dir, f"{symbol}-NEB.traj")

    if mat_status.get("neb_done", False) and os.path.exists(energies_file):
        print(f"{symbol}: NEB previously completed, reading barrier from file.")
        data = np.loadtxt(energies_file, comments="#")
        energies = data[:, 1]
        ref = min(energies[0], energies[-1])
        barrier = energies.max() - ref
        print(f"{symbol}: diffusion barrier ≈ {barrier:.3f} eV")
        return barrier

    # build NEB images
    images = [initial_relaxed]
    images += [initial_relaxed.copy() for _ in range(neb_n_images - 2)]
    images += [final_relaxed]

    neb = NEB(images, climb=True)
    neb.interpolate()

    for i, img in enumerate(images):
        img.calc = make_gpaw(f"{symbol}-NEB-img{i}")

    print(f"{symbol}: starting NEB optimization with {len(images)} images "
          f"(target fmax = {neb_fmax:.3f} eV/Å)")

    opt = FIRE(
        neb,
        logfile=os.path.join(output_dir, f"{symbol}-NEB.log"),
        trajectory=neb_traj_path,
    )

    step = 0
    t0 = time.time()
    for _ in opt.irun(fmax=neb_fmax, steps=neb_max_steps):
        step += 1
        elapsed = time.time() - t0
        try:
            neb_forces = neb.get_forces()
            maxf = float(np.abs(neb_forces).max())
        except Exception:
            maxf = float("nan")
        print(f"[{symbol} NEB] step {step:4d}, NEB max |F| = {maxf:.3f} eV/Å, elapsed = {elapsed:7.1f} s")

    total = time.time() - t0
    print(f"[{symbol} NEB] optimization finished after {step} steps, total time = {total:.1f} s.")

    # Energies and barrier
    energies = np.array([img.get_potential_energy() for img in images])
    ref = min(energies[0], energies[-1])
    barrier = energies.max() - ref

    print(f"{symbol}: diffusion barrier ≈ {barrier:.3f} eV")

    with open(energies_file, "w") as f:
        f.write("# image   energy_eV\n")
        for i, E in enumerate(energies):
            f.write(f"{i:3d}  {E:15.8f}\n")

    mat_status["neb_done"] = True
    status[symbol] = mat_status
    save_status(status)

    return barrier

print("NEB driver defined.")


NEB driver defined.


In [9]:
# 8. Run the Li(110) step-edge NEB (restartable)

barrier = run_Li110_step_neb()
print("Final reported barrier (eV):", barrier)



Running NEB for Li: Li(110) step-edge descent
Current status: {'initial_relaxed': False, 'final_relaxed': False, 'neb_done': False}
Li: loading existing unrelaxed initial/final from traj.
Li: relaxing initial adatom configuration...
[Li-initial] Restarting from existing trajectory: Li110_step_edge_neb_restartable_json/Li-initial.traj
[Li-initial] Failed to read existing traj (Empty file: Li110_step_edge_neb_restartable_json/Li-initial.traj), starting from input geometry.
[Li-initial] step    1, max |F| = 1.131 eV/Å, elapsed =    88.8 s
[Li-initial] step    2, max |F| = 1.030 eV/Å, elapsed =   121.1 s
[Li-initial] step    3, max |F| = 0.252 eV/Å, elapsed =   183.5 s
[Li-initial] step    4, max |F| = 0.344 eV/Å, elapsed =   242.3 s
[Li-initial] step    5, max |F| = 0.549 eV/Å, elapsed =   292.5 s
[Li-initial] step    6, max |F| = 0.661 eV/Å, elapsed =   343.7 s
[Li-initial] step    7, max |F| = 0.742 eV/Å, elapsed =   393.7 s
[Li-initial] step    8, max |F| = 0.723 eV/Å, elapsed =   446